# 4.2 - Macrobond Data API for Python - Building Backtesting Dataframe

*Using Macrobond Data API for Python to query revision history for one or more time series and return only vintage series relevant to a list of inputed rebalance dates. Given a list of rebalance dates for a backtesting model, this script identifies the closest previous vintage series for each date, and returns these series in a unified dataframe. The script has been optimized to handle one or many series requests for the same rebalance dates, and to return the outputed dataframes in a dictionary of dataframes*

This notebook aims to provide an example of how to use methods from Macrobond Data API for Python to work with Revision History and build out a dataframe for backtesting.

You can find a full description of all methods and parameters used in this example in the [documentation of the API](https://macrobond.github.io/macrobond-data-api/common/api.html).

***

## Importing Packages

In [1]:
import macrobond_data_api as mda
import pandas as pd

from datetime import datetime
from dateutil.tz import tzutc

pd.set_option('display.max_rows', None)

***
### Define Ticker Universe

Define tickers of interest and rebalance dates in tzutc format. Use get_revision_info() to return all possible vintage timestamps for each series, formatting dates in dataframe

In [2]:
tickers = ['krnaac0145', 'zanaac0004']
rebalance_dates = [datetime(2019, 6, 1, tzinfo=tzutc()), datetime(2019, 7, 1, tzinfo=tzutc()), datetime(2021,8, 1, tzinfo=tzutc()), datetime(2021,9, 1, tzinfo=tzutc()), datetime(2021,10, 1, tzinfo=tzutc()), datetime(2021,11, 1, tzinfo=tzutc()), datetime(2021, 12, 1, tzinfo=tzutc()), datetime(2022, 1, 1, tzinfo=tzutc())] 

data_frames = [mda.get_revision_info(tickers[n])[0].to_pd_data_frame() for n in range(len(tickers))] 
data_frames=[data_frames[n].groupby(data_frames[n].columns.to_list()[:-2])["vintage_time_stamps"].agg(list).reset_index() for n in range(len(data_frames))]
timestamps = [pd.DataFrame(data_frames[n].loc[0]['vintage_time_stamps'], columns=['vintage_time_stamp']) for n in range(len(data_frames))]

### Get all vintage series for each ticker of interest

Use mda.get_all_vintage_series() to return all possible vintages for a ticker.

In [3]:
vint_dfs_full = [mda.get_all_vintage_series(ticker).to_pd_data_frame().set_index('date') for ticker in tickers]

Adjusting column names for error handling

In [4]:
vint_dfs_full=[vint_dfs_full[n].rename(columns={vint_dfs_full[n].columns[0]:'pre_vintage'}) for n in range(len(vint_dfs_full))] 

### Creating list of relevant vintage timestamps for each inputed rebalance date. 

Each inputed rebalanced date is matched with the closest, previous vintage timestamp available for the series. 
We include error handling for situations where the inputed rebalance date is earlier than all existing vintage timestamps for a series. In this case, a Warning issued and the earliest record of a series is used as the nearest, previous vintage series. 

In [5]:
vint_ts_list = []

for n in range(len(timestamps)):
    temp_list = []
    for rebalance_date in rebalance_dates:
        if timestamps[n]['vintage_time_stamp'].loc[0]>rebalance_date:   # In case where inputed rebalance dates is earlier than all available vintage timestamps for a series, warning is issued and first release of the series is used for the vintage data
            print('Warning: The {} rebalance date is earlier than all vintages available for the series, \033[1m\x1B[3m{}\033[0;0m\x1B[0m, where the earliest vintage timestamp available is {}. The first release will be used for this reblanace date.'.format(rebalance_date, tickers[n], timestamps[n]['vintage_time_stamp'].loc[0]))
            temp_list.append(vint_dfs_full[n].columns[0])
        else: # In case where vintage timestamps exist previous to the rebalance date, the most recent previous vintage timeseries is selected as the vintage data for this rebalance date.
            temp_list.append(timestamps[n]['vintage_time_stamp'].loc[timestamps[n]['vintage_time_stamp']<rebalance_date].iloc[-1])
    vint_ts_list.append(temp_list)

### Return relevant vintage data

Filter full vintage dataframe to include only vintage timsestamps identified with the rebalance dates above.

In [6]:
final_dfs = [vint_dfs_full[n][vint_ts_list[n]] for n in range(len(vint_dfs_full))]

Rename columns with rebalance dates for formatting

In [7]:
for df in final_dfs:
    df.columns=rebalance_dates

### Creating dataframe dictionary

Storing backtest dataframes in dictionary to link with relevant ticker

In [8]:
vint_dfs_dict = {}
for i in range(len(tickers)):
    vint_dfs_dict[tickers[i]]=final_dfs[i]

for key in range(len(list(vint_dfs_dict.keys()))):
    print('{}:'.format(list(vint_dfs_dict.keys())[key]))
    display(vint_dfs_dict[list(vint_dfs_dict.keys())[key]])

krnaac0145:


,2019-06-01 00:00:00+00:00,2019-07-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,2021-09-01 00:00:00+00:00,2021-10-01 00:00:00+00:00,2021-11-01 00:00:00+00:00,2021-12-01 00:00:00+00:00,2022-01-01 00:00:00+00:00
date,,,,,,,,
1960-01-01,NaN,NaN,7.059600e+12,7.059600e+12,7.059600e+12,7.059600e+12,7.059600e+12,7.059600e+12
1960-04-01,NaN,NaN,7.377000e+12,7.377000e+12,7.377000e+12,7.377000e+12,7.377000e+12,7.377000e+12
1960-07-01,NaN,NaN,7.295000e+12,7.295000e+12,7.295000e+12,7.295000e+12,7.295000e+12,7.295000e+12
1960-10-01,NaN,NaN,7.338500e+12,7.338500e+12,7.338500e+12,7.338500e+12,7.338500e+12,7.338500e+12
1961-01-01,NaN,NaN,7.573400e+12,7.573400e+12,7.573400e+12,7.573400e+12,7.573400e+12,7.573400e+12
1961-04-01,NaN,NaN,7.900300e+12,7.900300e+12,7.900300e+12,7.900300e+12,7.900300e+12,7.900300e+12
1961-07-01,NaN,NaN,7.756100e+12,7.756100e+12,7.756100e+12,7.756100e+12,7.756100e+12,7.756100e+12
1961-10-01,NaN,NaN,7.856700e+12,7.856700e+12,7.856700e+12,7.856700e+12,7.856700e+12,7.856700e+12
1962-01-01,NaN,NaN,8.013200e+12,8.013200e+12,8.013200e+12,8.013200e+12,8.013200e+12,8.013200e+12


zanaac0004:


,2019-06-01 00:00:00+00:00,2019-07-01 00:00:00+00:00,2021-08-01 00:00:00+00:00,2021-09-01 00:00:00+00:00,2021-10-01 00:00:00+00:00,2021-11-01 00:00:00+00:00,2021-12-01 00:00:00+00:00,2022-01-01 00:00:00+00:00
date,,,,,,,,
1960-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.173810e+11
1960-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.307100e+11
1960-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.460170e+11
1960-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.543680e+11
1961-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.605640e+11
1961-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.568040e+11
1961-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.759420e+11
1961-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.839040e+11
1962-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.067670e+11
